In [34]:
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from typing import TypedDict
from dotenv import load_dotenv
import pprint
import json

In [25]:
load_dotenv()  # Load environment variables from .env file

True

In [26]:
model = ChatGroq(model="llama-3.1-8b-instant")

In [27]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    feedback: str
    grades: dict

In [28]:
def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state['title']

    # call llm gen outline
    prompt = f'Generate a detailed outline for a blog on the topic - {title}'
    outline = model.invoke(prompt).content

    # update state
    state['outline'] = outline

    return state

In [29]:
def Create_blog(state: BlogState) -> BlogState:
    title = state["title"]
    outline = state["outline"]
    prompt = f"Write a blog post based on the title '{title}' and the following outline:\n{outline}"
    content = model.invoke(prompt).content
    state["content"] = content
    return state

In [30]:
def evaluate_blog(state: BlogState) -> BlogState:
    content = state["content"]
    prompt = f"""
    Evaluate the following blog post for the following criteria:
    - Clarity
    - Engagement
    - Informativeness

    For each criterion, give a numeric score from 1 to 10.
    Then provide an overall written feedback.

    Blog content:
    {content}

    Return your response strictly in this JSON format:
    {{
        "grades": {{
            "clarity": <1-10>,
            "engagement": <1-10>,
            "informativeness": <1-10>
        }},
        "feedback": "<Your detailed written feedback here>"
    }}
    """

    response = model.invoke(prompt).content.strip()

    # Try parsing structured JSON response
    try:
        data = json.loads(response)
        state["grades"] = data.get("grades", {})
        state["feedback"] = data.get("feedback", "")
    except json.JSONDecodeError:
        # Fallback if the LLM gives non-JSON output
        state["grades"] = {}
        state["feedback"] = response

    return state

In [32]:
graph = StateGraph(BlogState)

graph.add_node("Create_outline", create_outline)
graph.add_node("Create_blog", Create_blog)
graph.add_node("Evaluate_blog", evaluate_blog)

graph.add_edge(START, "Create_outline")
graph.add_edge("Create_outline", "Create_blog")
graph.add_edge("Create_blog", "Evaluate_blog")
graph.add_edge("Evaluate_blog", END)

app = graph.compile()

In [35]:
initial_state = {"title": "The Future of AI in Everyday Life"}
final_state = app.invoke(initial_state)

print("\n📰 Blog Title:", final_state["title"])
print("\n📊 Grades:", final_state["grades"])
print("\n💬 Editor Feedback:\n", final_state["feedback"])
print("\n📝 Blog Preview:\n", final_state["content"][:700], "...")



📰 Blog Title: The Future of AI in Everyday Life

📊 Grades: {}

💬 Editor Feedback:
 {
  "grades": {
    "clarity": 9,
    "engagement": 8,
    "informativeness": 9
  },
  "feedback": "The blog post presents a comprehensive overview of the future of AI in everyday life, covering various aspects such as home automation, healthcare, transportation, education, job market, social interactions, and ethical considerations. The writing is clear and concise, making it easy to follow and understand the concepts. The structure is well-organized, with each section providing a clear and concise overview of the topic.

The engagement score is lower due to the lack of personal anecdotes, examples, or real-life scenarios that might make the content more relatable and interesting to the readers. However, the use of technical terms and concepts is well-explained, making it accessible to a general audience.

The informativeness score is high due to the thorough coverage of the topic. The author provides 